# Copter model derivation

In [15]:
import sympy
from model import *

sympy.init_printing()

In [24]:
# Thrust, mass, drag coeff
T, m, cd = sympy.symbols("T m c_d")

# Torque vector
taux, tauy, tauz = sympy.symbols("tau_x tau_y tau_z")
tau = sympy.Matrix([taux, tauy, tauz])

# Simplified inertia matrix
Ixx, Iyy, Izz = sympy.symbols("I_xx I_yy I_zz")
I = sympy.Matrix([[Ixx, 0, 0], [0, Iyy, 0], [0, 0, Izz]])

In [27]:
# Predicted acceleration = gravity + thrust - air drag
# Assuming thrust is always generated in the local UP direction
ap = gv + rot_v(UP * (T/m), q) - v * (cd/m)

# Predicted angular acceleration
aap = I**-1 * (tau - w.cross(I * w))

sympy.simplify(ap), sympy.simplify(aap)

⎛⎡       2⋅T⋅q_w⋅q_y + 2⋅T⋅qₓ⋅q_z - c_d⋅vₓ        ⎤                            ↪
⎜⎢       ─────────────────────────────────        ⎥  ⎡I_yy⋅ω_y⋅ω_z - I_zz⋅ω_y⋅ ↪
⎜⎢                       m                        ⎥  ⎢──────────────────────── ↪
⎜⎢                                                ⎥  ⎢              Iₓₓ        ↪
⎜⎢      -2⋅T⋅q_w⋅qₓ + 2⋅T⋅q_y⋅q_z - c_d⋅v_y       ⎥  ⎢                         ↪
⎜⎢      ───────────────────────────────────       ⎥  ⎢-Iₓₓ⋅ωₓ⋅ω_z + I_zz⋅ωₓ⋅ω_ ↪
⎜⎢                       m                        ⎥, ⎢──────────────────────── ↪
⎜⎢                                                ⎥  ⎢             I_yy        ↪
⎜⎢     2       2        2        2                ⎥  ⎢                         ↪
⎜⎢T⋅q_w  - T⋅qₓ  - T⋅q_y  + T⋅q_z  - c_d⋅v_z - g⋅m⎥  ⎢ Iₓₓ⋅ωₓ⋅ω_y - I_yy⋅ωₓ⋅ω_ ↪
⎜⎢────────────────────────────────────────────────⎥  ⎢ ─────────────────────── ↪
⎝⎣                       m                        ⎦  ⎣              I_zz       ↪

↪          ⎞
↪ ω_z + τₓ⎤⎟
↪

In [28]:
# Derivative (jacobian) of acceleration with respect to state variables
ap.jacobian(v), ap.jacobian(q.to_Matrix())

⎛⎡-c_d               ⎤  ⎡2⋅T⋅q_y    2⋅T⋅q_z    2⋅T⋅q_w   2⋅T⋅qₓ ⎤⎞
⎜⎢─────    0      0  ⎥  ⎢───────    ───────    ───────   ────── ⎥⎟
⎜⎢  m                ⎥  ⎢   m          m          m        m    ⎥⎟
⎜⎢                   ⎥  ⎢                                       ⎥⎟
⎜⎢       -c_d        ⎥  ⎢-2⋅T⋅qₓ   -2⋅T⋅q_w    2⋅T⋅q_z   2⋅T⋅q_y⎥⎟
⎜⎢  0    ─────    0  ⎥, ⎢────────  ─────────   ───────   ───────⎥⎟
⎜⎢         m         ⎥  ⎢   m          m          m         m   ⎥⎟
⎜⎢                   ⎥  ⎢                                       ⎥⎟
⎜⎢              -c_d ⎥  ⎢2⋅T⋅q_w   -2⋅T⋅qₓ    -2⋅T⋅q_y   2⋅T⋅q_z⎥⎟
⎜⎢  0      0    ─────⎥  ⎢───────   ────────   ─────────  ───────⎥⎟
⎝⎣                m  ⎦  ⎣   m         m           m         m   ⎦⎠

In [29]:
# Derivative (jacobian) of angular acceleration with respect to state variables
sympy.simplify(aap.jacobian(w))

⎡                   ω_z⋅(I_yy - I_zz)  ω_y⋅(I_yy - I_zz)⎤
⎢        0          ─────────────────  ─────────────────⎥
⎢                          Iₓₓ                Iₓₓ       ⎥
⎢                                                       ⎥
⎢ω_z⋅(-Iₓₓ + I_zz)                     ωₓ⋅(-Iₓₓ + I_zz) ⎥
⎢─────────────────          0          ──────────────── ⎥
⎢      I_yy                                  I_yy       ⎥
⎢                                                       ⎥
⎢ω_y⋅(Iₓₓ - I_yy)    ωₓ⋅(Iₓₓ - I_yy)                    ⎥
⎢────────────────    ───────────────           0        ⎥
⎣      I_zz               I_zz                          ⎦